# Interacting with a CSV Data

## Setup and Connect to the Hugging Face Inference API
**Note**: Model utilized here is `deepseek-ai/DeepSeek-R1-Distill-Qwen-32B`.You must replace `YOUR_HUGGINGFACE_API_KEY` with your own Hugging Face API key.

In [2]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from typing import Optional, List, Generator, Any

In [3]:
from langchain.llms.base import LLM
from huggingface_hub import InferenceClient

## Authenticate with Hugging Face

In [4]:
# Load the environment variables
load_dotenv()

# Access the API key from the environment
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

In [5]:
client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

## Load the dataset

In [6]:
df = pd.read_csv("all-states-history.csv").fillna(value = 0)

## Prepare the Langchain dataframe agent

In [7]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

##  Create a Custom LLM Class

In [8]:
class HuggingFaceStreamingLLM(LLM):
    """A custom LLM that wraps Hugging Face streaming completions with structured output parsing."""
    model_name: str = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
    temperature: float = 0.7
    max_tokens: int = 2048
    top_p: float = 0.7

    @property
    def _llm_type(self) -> str:
        return "huggingface_streaming_llm"

    def _parse_response_content(self, content: str) -> str:
        """Parse model output to extract either action steps or final answer."""    
        # Remove XML-like tags first
        cleaned_content = re.sub(r"<\/?think>", "", content)
    
        # Add code block detection
        code_match = re.search(r"```python\n(.*?)\n```", content, re.DOTALL)
        if code_match:
            return f"Action: python_repl_ast\nAction Input: {code_match.group(1).strip()}"
            
        # Try to extract Action/Action Input pattern
        action_match = re.search(
            r"Action: (.*?)\nAction Input: (.*?)(\n|$)", 
            content, 
            re.DOTALL
        )
        if action_match:
            return f"Action: {action_match.group(1).strip()}\nAction Input: {action_match.group(2).strip()}"
        
        # Try to extract Final Answer pattern
        final_answer_match = re.search(
            r"Final Answer: (.*?)(\n|$)", 
            content, 
            re.DOTALL
        )
        if final_answer_match:
            return f"Final Answer: {final_answer_match.group(1).strip()}"
        
        # Fallback: Return the original content with 'Final Answer' wrapper
        return f"Final Answer: {content.strip()}"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Synchronously call the model and return parsed output."""
        messages = [{"role": "user", "content": prompt}]
        
        result = client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            top_p=self.top_p,
            stream=False
        )

        raw_content = result["choices"][0]["message"]["content"]
        
        parsed = self._parse_response_content(raw_content)
        
        return parsed


## Define Agent

In [9]:
# Instantiate Custom LLM
llm = HuggingFaceStreamingLLM()

# Create the agent
agent = create_pandas_dataframe_agent(llm=llm, df=df, return_intermediate_steps=True,verbose=True,allow_dangerous_code=True)

In [10]:
# Invoke the agent with a query
result = agent.invoke("how many rows are there?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action: python_repl_ast
Action Input: df.shape(20780, 41)Final Answer: There are 20,780 rows in the DataFrame.

> Finished chain.


In [11]:
# Invoke the agent with a query
result = agent.invoke("how many features are there in dataframe?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action: python_repl_ast
Action Input: print(df.shape[1])41
Final Answer: The dataframe has 41 features.

> Finished chain.


In [12]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
**REQUIRED FORMAT**
Thought: Explain your approach
Action: python_repl_ast
Action Input: Code to calculate answer
Observation: [code output]
Final Answer: [Only after 2 consistent methods]

**STRICT RULES**
1. Strictly, if no code executed, say "I need to calculate this"
2. perform Calculations using Action and Action input
2. If columns missing, list available columns
3. Perform date filtering, validation
4. Compare min/max vs last/first values

- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

In [15]:
QUESTION = "How may patients were hospitalized during Mar 2021 in Alaska use column hospitalizedCumulative" 

result = agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action: python_repl_ast
Action Input: print(df.columns)Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncreas